In [1]:
import os

data_dir = '/kaggle/input/curated-chest-xray-image-dataset-for-covid19/Curated X-Ray Dataset'

# Obtener una lista de las clases (nombres de las carpetas)
classes = os.listdir(data_dir)

# Imprimir la cantidad de imágenes en cada clase
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    num_images = len(os.listdir(class_dir))
    print(f'Clase: {class_name}, Cantidad de Imágenes: {num_images}')

Clase: Normal, Cantidad de Imágenes: 3270
Clase: COVID-19, Cantidad de Imágenes: 1281
Clase: Pneumonia-Viral, Cantidad de Imágenes: 1656
Clase: Pneumonia-Bacterial, Cantidad de Imágenes: 3001


In [2]:
import os
import numpy as np
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ruta de la carpeta original con las imágenes
data_dir = '/kaggle/input/curated-chest-xray-image-dataset-for-covid19/Curated X-Ray Dataset'

# Directorio donde se guardarán las imágenes aumentadas
augmented_dir = '/kaggle/working/augmented_data'

# Crea el directorio si no existe
if not os.path.exists(augmented_dir):
    os.makedirs(augmented_dir)

# Definir las clases basadas en las carpetas dentro de data_dir
classes = ['COVID-19', 'Normal', 'Pneumonia-Bacterial', 'Pneumonia-Viral']

# Obtener la cantidad máxima de imágenes entre todas las clases
max_images_per_class = max([len(os.listdir(os.path.join(data_dir, class_name))) for class_name in classes])

# Crear un generador de aumento de datos para todas las clases
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Crear un directorio temporal para las imágenes aumentadas
temp_augmented_dir = os.path.join(augmented_dir, 'temp')
os.makedirs(temp_augmented_dir, exist_ok=True)

# Generar imágenes aumentadas y guardarlas en el directorio temporal
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    num_images = len(os.listdir(class_dir))

    # Si la clase tiene menos imágenes que la cantidad máxima, generar imágenes aumentadas
    if num_images < max_images_per_class:
        target_dir = os.path.join(temp_augmented_dir, class_name)
        os.makedirs(target_dir, exist_ok=True)
        
        image_files = [os.path.join(class_dir, img_name) for img_name in os.listdir(class_dir)]
        image_generator = datagen.flow_from_directory(
            data_dir,
            target_size=(224, 224),
            batch_size=32,
            classes=[class_name],
            save_to_dir=target_dir,
            save_prefix='aug',
            save_format='jpeg'
        )
        num_augmented_images = max_images_per_class - num_images  # Cuántas imágenes adicionales necesitamos
        for _ in range(num_augmented_images // 32):  # Aumentar en lotes de 32 imágenes
            image_batch, _ = next(image_generator)

# Crear un nuevo directorio equilibrado y copiar todas las imágenes originales y aumentadas
balanced_data_dir = '/kaggle/working/balanced_data'
os.makedirs(balanced_data_dir, exist_ok=True)

for class_name in classes:
    source_dir = os.path.join(data_dir, class_name)
    target_dir = os.path.join(balanced_data_dir, class_name)

    # Crear el directorio de destino si no existe
    os.makedirs(target_dir, exist_ok=True)

    # Copiar todas las imágenes originales
    for img_name in os.listdir(source_dir):
        source_path = os.path.join(source_dir, img_name)
        target_path = os.path.join(target_dir, img_name)
        shutil.copy(source_path, target_path)

    # Si es la clase con menos imágenes, copiar también las imágenes aumentadas
    if class_name in os.listdir(temp_augmented_dir):
        temp_class_dir = os.path.join(temp_augmented_dir, class_name)
        for img_name in os.listdir(temp_class_dir):
            source_path = os.path.join(temp_class_dir, img_name)
            target_path = os.path.join(target_dir, img_name)
            shutil.copy(source_path, target_path)

# Imprimir la cantidad de imágenes en cada clase después del aumento y el balanceo
for class_name in classes:
    num_images_after_augmentation = len(os.listdir(os.path.join(balanced_data_dir, class_name)))
    print(f'Clase: {class_name}, Cantidad de Imágenes después del Aumento y Balanceo: {num_images_after_augmentation}')


Found 1281 images belonging to 1 classes.
Found 3001 images belonging to 1 classes.
Found 1656 images belonging to 1 classes.
Clase: COVID-19, Cantidad de Imágenes después del Aumento y Balanceo: 3234
Clase: Normal, Cantidad de Imágenes después del Aumento y Balanceo: 3270
Clase: Pneumonia-Bacterial, Cantidad de Imágenes después del Aumento y Balanceo: 3257
Clase: Pneumonia-Viral, Cantidad de Imágenes después del Aumento y Balanceo: 3256


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Ruta del directorio con imágenes balanceadas
balanced_data_dir = '/kaggle/working/balanced_data'

# Parámetros
batch_size = 32
epochs = 8
input_shape = (224, 224, 3)

# Crear generador de datos para aumento y validación
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
)

# Cargar datos y dividir en conjuntos de entrenamiento y validación
train_generator = datagen.flow_from_directory(
    balanced_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    balanced_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Crear modelo ResNet50
base_model = ResNet50(weights='/kaggle/input/tf-keras-pretrained-model-weights/No Top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=input_shape)
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Gráfico de pérdida y precisión
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.show()

# Evaluación del modelo en el conjunto de prueba
test_generator = datagen.flow_from_directory(
    balanced_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Loss: {test_loss:.4f}')

# Matriz de confusión y reporte de clasificación
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = list(train_generator.class_indices.keys())
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

# Guardar pesos y modelo
model.save('/kaggle/working/resnet50_model.h5')


Found 10415 images belonging to 4 classes.
Found 2602 images belonging to 4 classes.
Epoch 1/8
326/326 [==============================] - 5612s 17s/step - loss: 0.7614 - accuracy: 0.7858 - val_loss: 374.4056 - val_accuracy: 0.2483
Epoch 2/8
326/326 [==============================] - 4985s 15s/step - loss: 0.4468 - accuracy: 0.8479 - val_loss: 2.3855 - val_accuracy: 0.3128
Epoch 3/8
326/326 [==============================] - 5562s 17s/step - loss: 0.2995 - accuracy: 0.8837 - val_loss: 1.0053 - val_accuracy: 0.6030
Epoch 4/8
326/326 [==============================] - 4863s 15s/step - loss: 0.1982 - accuracy: 0.9241 - val_loss: 0.3797 - val_accuracy: 0.8413
Epoch 5/8
326/326 [==============================] - 5077s 16s/step - loss: 0.1284 - accuracy: 0.9543 - val_loss: 0.3979 - val_accuracy: 0.8716
Epoch 6/8
326/326 [==============================] - 5406s 17s/step - loss: 0.0817 - accuracy: 0.9718 - val_loss: 0.5199 - val_accuracy: 0.8559
Epoch 7/8
326/326 [==============================